In [13]:
import numpy as np

In [4]:
# For getting how long each group of customers would spend
np.random.uniform(30,150) # Generate one random number between 30 and 150 under uniform distribution

144.19198397661827

In [5]:
class Tables:
    def __init__(self, num, size):
        self.num=num # No. of the table
        self.size = size # Size of the table: for group of up to 2, 4 or 6.
        self.currentStatus = None # Is the table occupied or not.
        self.time_remaining = 0 
        
    def busy(self):
        if self.currentStatus != None:
            return True
        else:
            return False
    
    def startNext(self, newtask):
        self.currentStatus = newtask
        self.time_remaining = newtask.time_needs() # here maybe generate a random time from 10 - 60 mins using generat

In [33]:
def table_generator(number_tables_2, number_tables_4, number_tables_6):
    table_2_list = []
    table_4_list = []
    table_6_list = [] 
    for i in range(number_tables_2):
        table_2_list.append(Tables(i,2))
    
    for i in range(number_tables_4):
        table_4_list.append(Tables(i+number_tables_2,4))
    
    for i in range(number_tables_6):
        table_6_list.append(Tables(i+number_tables_4+number_tables_6,6))
    
    return (table_2_list, table_4_list, table_6_list)

In [38]:
table_2_list, table_4_list, table_6_list = table_generator(6,4,2)

In [39]:
table_2_list[0].num

0

In [40]:
table_4_list[0].num

2

In [70]:
class Groups:
    def __init__(self, time, size, vip):
        self.timestamp = time # Time when group registered (entered into the queue)
        self.size = size #randomly define size from 1 - 6
        self.vip = vip # Whether the group is a vip group
        self.table=None # Which table the group will be assigned to
    
    def get_stamp(self):
        return self.timestamp
    
    def get_size(self):
        return self.size
    
    def wait_time(self, current_time):
        return current_time - self.timestamp
    
    def get_vip(self):
        return self.vip

In [78]:
class Queue:
    def __init__(self):
        self.groups=[]
    
    def queue_size(self):
        return len(self.groups)
    
    def isEmpty(self):
        if len(self.groups)>0:
            return True
        else:
            False
    
    def add_queue(self, group):
        if group.get_vip():
            self.groups.insert(4,group) # If current group is a VIP group, move it forward by four groups
        else:
            self.groups.insert(0, group)
        
    
    def del_queue(self): #delete last=delete first come group 
        self.groups.pop()

In [26]:
def mod_pert_random(low, likely, high, confidence=4, samples=30):
    """Produce random numbers according to the 'Modified PERT' 
    distribution. 
    
    :param low: The lowest value expected as possible.
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature 
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        
    Formulas from "Modified Pert Simulation" by Paulo Buchsbaum.
    """
    # Check minimum & maximum confidence levels to allow:
    confidence = min(8, confidence)
    confidence = max(2, confidence)

    mean = (low + confidence * likely + high)/(confidence + 2)

    a = (mean - low)/(high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)
    
    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    return beta

In [79]:
def newGroup(Duration):
    size = np.random.randint(1,7,30)
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    counter = 0
    group_2 = Queue()
    group_4 = Queue()
    group_6 = Queue()
    for i in range(Duration):
        if i in timestamp_list:
            if size[counter] == 1 or size[counter] == 2:
                group_2.add_queue(Groups(i, 2, vip[counter]))
                counter += 1
            elif size[counter] == 3 or size[counter] == 4:
                group_4.add_queue(Groups(i, 4, vip[counter]))
                counter += 1
            else:
                group_6.add_queue(Groups(i, 6, vip[counter]))
                counter += 1
    return (group_2, group_4 , group_6)

In [80]:
group_2, group_4, group_6 = newGroup(240)

# 1. test version 1 - Do Not Run

In [1]:
def generation(Duration):
    # Generate size 
    size = np.random.randint(1,7,30)
    print(size)
    
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    
    print(timestamp_list)
    
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    
    table_2,table_4,table_6=tablesSetting(6,4,2)
    
    total_timeR = [0]
    nextGroup_endTime_2={}
    
    # Add every group into corresponding queue
    groupNumb_2=0
    groupNumb_4=0
    groupNumb_6=0
    
    for i in range(Duration):
        if i in timestamp_list:
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter], groupNumb_2))
                counter += 1
                groupNumb_2+=1
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter], groupNumb_4))
                counter += 1
                groupNumb_4+=1
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter], groupNumb_6))
                counter += 1
                groupNumb_6 += 1
                
                
        
    for i in range(Duration):   
        if i in nextGroup_endTime_2.values():
            for num in nextGroup_endTime_2.keys():
                if i == nextGroup_endTime_2[num]:
                    print('Clean:', num.num,'\n')
                    table_2[num.num].currentGroup = None
            #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
            
        
        if (len(queue_2.groups)>0):
            current_q=[]
            for item in queue_2.groups:
                current_q.append(item.get_groupID())
            print(current_q)
            if (i >= (queue_2.groups[len(queue_2.groups)-1].get_stamp())):
                for t in table_2:
                    if (len(queue_2.groups)>0) and (not t.busy()) and (i>=(queue_2.groups[len(queue_2.groups)-1].get_stamp())):
                        print('Head GroupID:',queue_2.groups[len(queue_2.groups)-1].get_groupID())
                        nextGroup = queue_2.del_queue()
                     
                        print('Wait time:', nextGroup.wait_time(i))
                     
                     
                     
                        t.startNext(nextGroup)
                        print('Group No.:', nextGroup.get_groupID())
                        print('VIP?:', nextGroup.get_vip())
                        print('Registration time:', nextGroup.get_stamp())
                        print('Entering into table time:',i)
                         
                         
                         
                        nextGroup.table=t.num
                        print('Table No.', t.num)
                         
                        nextGroup_endTime_2[t] = i+nextGroup.timeRequest+2  #2min for table cleaning, rearragement and the time to walk in blabla
                        print('\n')
                        total_timeR.append(nextGroup.timeRequest+2)
        
    avg_timeR = sum(total_timeR)/len(total_timeR)
        
    return (queue_2, queue_4, queue_6)

# 2. test version 2 - Do Not Run

In [2]:
def generation(Duration):
    size = np.random.randint(1,3,90)  #1-6 people for one group, and generate x groups
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    table_2,table_4,table_6=tablesSetting(6,4,2)  #setting tables
    
    #print(table_2,table_4,table_6)
    total_timeR = []
    nextGroup_endTime_2={}
    
    groupNumb_2=1
    groupNumb_4=1
    groupNumb_6=1
    
    for i in range(Duration):
        #print(i,'/n')
        if i in timestamp_list:
            #print(size[counter],'**')
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter],groupNumb_2))
                
                print('new regislation vip+No+time: ',vip[counter], groupNumb_2,i)
                
                counter += 1
                groupNumb_2 += 1
                
                current_q=[]
                for item in queue_2.groups:
                    current_q.append(item.get_groupID())
                print(current_q,'current queue_2')
                
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter],groupNumb_4))
                counter += 1
                groupNumb_4 += 1
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter],groupNumb_6))
                counter += 1
                groupNumb_6 += 1
                
        
        if i in nextGroup_endTime_2.values():
            print('\nkeys',nextGroup_endTime_2.keys())
            print('values',nextGroup_endTime_2.values())
            for n in list(nextGroup_endTime_2.keys()):
                if i == int(nextGroup_endTime_2[n]):
                    print('Clean:', n,'\n')
                    table_2[n-1].currentGroup = None
            #table_2[对应nextGroup_endTime_2值为i的keys].currentGroup = None  
            #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
                    
        
        for t in table_2:
            if (not t.busy()) and (not queue_2.isEmpty()):
                
                
                print(t.get_tableNum(),'table ID')

                nextGroup = queue_2.del_queue()
                print(nextGroup.get_stamp(),'regislation time')
                print(nextGroup.wait_time(i),'wait time')
                t.startNext(nextGroup)
                print('Group can sit now: ',nextGroup.get_groupID())
                nextGroup_endTime_2[t.get_tableNum()] = i+nextGroup.get_time_request()+2  #end time for each 2-people-table
                #2min for table cleaning, rearragement and the time to walk in blabla
                print(nextGroup_endTime_2[t.get_tableNum()],'next group end time')
                total_timeR.append(int(nextGroup.get_time_request())+2)  #def generation(Duration):
                print(total_timeR,'total request time \n')

        
    #avg_timeR = sum(total_timeR)/len(total_timeR)
        

In [6]:
for i in range(4):
    print(i+6)

6
7
8
9


In [7]:
241 // 2

120

In [32]:
time_2=[1,2,3,4,5,5,5,6,6,7,8,9]
time=np.array(time_2)

In [36]:
time_3=list(time)

In [34]:
index = np.argwhere(time==6)
time=np.delete(time, index)

In [37]:
time_3

[1, 2, 3, 4, 5, 5, 5, 7, 8, 9]

In [38]:
time_3.remove(5)

In [39]:
time_3

[1, 2, 3, 4, 5, 5, 7, 8, 9]